In [1]:
import cv2
import sys
from math import dist
import numpy as np
import os

In [2]:
base_dir = "./detect_data/test_videos/"             # Folder containing all the videos
labels_dir = "./detect_data/Results_YOLO_Mod/"      # Folder containing txt files generated by YOLOv7 in Challenge format for all the videos in base_dir
final_list = os.listdir(base_dir)
labels_list = os.listdir(labels_dir)
for i in range(len(final_list)):
    final_list[i] = os.path.join(base_dir, final_list[i])
    labels_list[i] = os.path.join(labels_dir, labels_list[i])
final_list.sort()
labels_list.sort()
len(final_list), len(labels_list)

(1, 1)

In [5]:
def e_dist(n_bbox, bboxs):
    x1, y1 = n_bbox[0], n_bbox[1]
    for i, bbox in enumerate(bboxs):
        x2, y2 = bbox[0] + bbox[2]//2, bbox[1] + bbox[3]//2
        # print(dist([x1, y1], [x2, y2]), [x1, y1], [x2, y2])
        if dist([x1, y1], [x2, y2]) < 500:
            return False
    return True

In [6]:
os.makedirs('./results_without_resnet', exist_ok=True)

In [7]:
for i in range(len(labels_list)):
    video_adr = final_list[i]
    bbox_adr = labels_list[i]
    
    trackers = []
    conf = []

    video = cv2.VideoCapture(video_adr)

    if not video.isOpened():
        print("Could not open video")
        sys.exit()

    ok, frame = video.read()
    if not ok:
        print ('Cannot read video file')
        sys.exit()
    
    video = cv2.VideoCapture(video_adr)

    write_file = open('./results_without_resnet/' + video_adr.split('/')[-1][:-3] + 'txt', 'w')

    bbox_file = open(bbox_adr,'r')

    index = 0

    bbox_file_array = []

    for l in bbox_file:
        bbox_file_array.append(l)
    prev_bbox = []

    for j, l in enumerate(bbox_file_array):
        # Read a new frame
        prev_frame = frame
        ok, frame = video.read()
        
        if not ok:
            break
        h, w = frame.shape[:2]
        
        bboxs = []
        line = l.split()

        # If YOLOv7 detected drones
        if(int(line[1]) != 0):
            for i in range(int(line[1])):
                bbox = line[i*5 + 2: (i+1)*5 + 2]
                n_bbox = list(map(float, bbox))
                t_conf = n_bbox[-1]
                n_bbox = n_bbox[:-1]
                n_bbox = list(map(int, n_bbox))
                bboxs.append(n_bbox)
            prev_bbox = bboxs
            
            # Clear trackers
            trackers = []
            conf = []

        # If YOLOv7 did not detect drones
        else:
            # If YOLOv7 detected drones in previous frame
            if(prev_bbox != []):
                n_bbox = prev_bbox
                for n in n_bbox:
                    tracker = cv2.legacy.TrackerCSRT_create()
                    if 0 <= n[0] and 0 <= n[2] and n[0] + n[2] <= prev_frame.shape[1] and 0 <= n[1] and 0 <= n[3] and n[1] + n[3] <= prev_frame.shape[0]:
                        tracker.init(prev_frame, n)
                        trackers.append(tracker)
                        conf.append(n_bbox[-1])
                        
            # Update trackers and predict bounding boxes
            for t in trackers:
                _, bbox = t.update(frame)
                bboxs.append(bbox)
            prev_bbox = []

        timer = cv2.getTickCount()

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

        # Draw bounding box
        if ok:
            write_str = f"{index} {len(bboxs)} "
            # Tracking success
            for idx, bbox in enumerate(bboxs):
                write_str += f"{int(bbox[0])} {int(bbox[1])} {int(bbox[2])} {int(bbox[3])} "
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
            write_file.write(write_str + '\n')
            
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

        
        # Display tracker type on frame
        cv2.putText(frame, "CSRT Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2)
        # Display result
        frame = cv2.resize(frame, (780, 540), interpolation = cv2.INTER_LINEAR)
        cv2.imshow("Tracking", frame)

        # Exit if ESC pressed
        if cv2.waitKey(1) & 0xFF == ord('q'): # if press SPACE bar
            break

        index += 1
    
    write_file.close()
    video.release()
    cv2.destroyAllWindows()

Could not open video


SystemExit: 

/home/ajeet/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
